# Import required libraries

In [1]:
import pandas as pd
import re
import numpy as np
import os

In [2]:
os.chdir('..')                        # to move in back directory (for importing config)
import config

# Batting Data

In [13]:
batting_df = pd.read_csv(config.BATTING_FILE_PATH)
batting_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [24]:
batting_df['dismissal'] = batting_df['dismissal'].apply(lambda x : 0 if x == 'not out ' else 1)        # lable encoding, 1 for out and 0 for not_out

In [27]:
batting_df['batsmanName'] = batting_df['batsmanName'].apply(lambda x : re.sub('â€|\xa0|\(c\)|\xa0†|†','',x))   # removed unwanted chrs from player names

In [30]:
batting_df[['battingPos','runs','balls','4s','6s']] = batting_df[['battingPos','runs','balls','4s','6s']].astype(int)   # converted data from object to int

In [31]:
batting_df['SR'] = batting_df['SR'].apply(lambda x : 0.00 if x == '-' else float(x))       # removed unwanted char '-' and converted all values to float

In [33]:
batting_df.head(2)

,batsmanName,dismissal,runs,balls,M,4s,6s,SR,match,contry,battingPos
0,Michael van Lingen,1,3,6,7,0,0,50.0,Namibia VS Sri Lanka,Namibia,1
1,Divan la Cock,1,9,9,15,1,0,100.0,Namibia VS Sri Lanka,Namibia,2


# Bowling Data

In [37]:
bowling_df = pd.read_csv(config.BOWLING_FILE_PATH)
bowling_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [39]:
bowling_df[['overs','maiden','runs','wickets','economy','0s','4s','6s','wides','noBalls']] = bowling_df[['overs','maiden','runs','wickets','economy','0s','4s','6s','wides','noBalls']].astype(float)              # data type casting to int

In [43]:
bowling_df['bowlerName'] = bowling_df['bowlerName'].apply(lambda x : re.sub('â€|\xa0|\(c\)|\xa0†|†','',x))    # data cleaning

In [40]:
bowling_df['complete_overs'] = bowling_df['overs'].apply(lambda x : int(str(x).split('.')[0]))                # conversion of over into obers and balls
bowling_df['balls_from_incomplete_over'] = bowling_df['overs'].apply(lambda x : int(str(x).split('.')[1]))

In [41]:
bowling_df['balls_bowled'] = bowling_df['complete_overs']*6 +bowling_df['balls_from_incomplete_over']

In [45]:
bowling_df.head(2)

,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,match,contry,complete_overs,balls_from_incomplete_over,balls_bowled
0,Maheesh Theekshana,4.0,0.0,23.0,1.0,5.75,7.0,0.0,0.0,2.0,0.0,Namibia VS Sri Lanka,Sri Lanka,4,0,24
1,Dushmantha Chameera,4.0,0.0,39.0,1.0,9.75,6.0,3.0,1.0,2.0,0.0,Namibia VS Sri Lanka,Sri Lanka,4,0,24


# Player Data

In [52]:
player_df = pd.read_csv(config.PLAYER_FILE_PATH)
player_df.drop('link', axis = 1, inplace = True)
player_df = player_df.rename({'Unnamed: 0':'name'},axis = 1)

In [54]:
player_df['name'] = player_df['name'].apply(lambda x : re.sub('†|â€|\xa0|\(c\)','',x))      # removed unwanted noise and (c)->caption symbol

In [57]:
player_df.head(2)

,name,contry,battingStyle,bowlingStyle,playingRole
0,Maheesh Theekshana,Sri Lanka,Right hand Bat,Right arm Offbreak,Bowler
1,Dushmantha Chameera,Sri Lanka,Right hand Bat,Right arm Fast,Bowler


# Derived Columns For Analysis

### assign indivisual runs to players

In [60]:
indivisual_runs_dict = batting_df.groupby('batsmanName')['runs'].sum().to_dict()
player_df['indivisual_runs'] = player_df['name'].map(indivisual_runs_dict)

### total balls played

In [61]:
total_ball_dict = batting_df.groupby('batsmanName')['balls'].sum().to_dict()
player_df['total_balls'] = player_df['name'].map(total_ball_dict)

### innings played

In [62]:
inning_dict = batting_df.loc[batting_df['balls']>0]['batsmanName'].value_counts().to_dict() # inning will be considered if only he faced at least 1 ball(i.e. greater than zero)
player_df['innings'] = player_df['name'].map(inning_dict)

### dismissed

In [63]:
diss_dict =  batting_df.groupby('batsmanName')['dismissal'].sum().to_dict()
player_df['dissmissals'] = player_df['name'].map(diss_dict)

### fours

In [64]:
fours_dict = batting_df.groupby('batsmanName')['4s'].sum().to_dict()
player_df['fours'] = player_df['name'].map(fours_dict)

### sixes

In [65]:
sixes_dict = batting_df.groupby('batsmanName')['6s'].sum().to_dict()
player_df['sixes'] = player_df['name'].map(sixes_dict)

### batting position

In [66]:
bat_pos_dict = batting_df.groupby('batsmanName')['battingPos'].mean().round().to_dict()
player_df['bat_pos'] = player_df['name'].map(bat_pos_dict)

### average runs

In [67]:
player_df['batting_Avg'] = np.around(player_df['indivisual_runs'] / player_df['dissmissals'],1)

### overall strike rate

In [68]:
player_df['str_rate'] = np.round(player_df['indivisual_runs'] / player_df['total_balls'] * 100,1)

### boundry percentage

In [69]:
player_df['boundry%'] = np.round(((player_df['fours']*4)+ (player_df['sixes']*6))/player_df['indivisual_runs'] * 100,1)

### Avg balls Faced

In [70]:
balls_dict = batting_df.groupby('batsmanName')['balls'].mean().to_dict()
player_df['avg_balls'] = np.around(player_df['name'].map(balls_dict),0)

### inning bowled

In [71]:
bolled_inning_dict = bowling_df['bowlerName'].value_counts().to_dict()
player_df['innings_bowled'] = player_df['name'].map(bolled_inning_dict)

### bowling economy 

In [72]:
eco_dict = bowling_df.groupby('bowlerName')['economy'].mean()
player_df['bow_economy'] = player_df['name'].map(eco_dict)

### Balls_bowled

In [73]:
bowled_dict = bowling_df.groupby('bowlerName')['balls_bowled'].sum()
player_df['balls_bowled'] = player_df['name'].map(bowled_dict)

### Wickets

In [74]:
wicket_dict = bowling_df.groupby('bowlerName')['wickets'].sum()
player_df['wickets'] = player_df['name'].map(wicket_dict)

### bowling Srike rate

In [75]:
player_df['Bow_strike_rate'] = np.round(player_df['balls_bowled'] / player_df['wickets'],1)

### Runs Consided

In [76]:
consided_dict = bowling_df.groupby('bowlerName')['runs'].sum()
player_df['runs_consided'] = player_df['name'].map(consided_dict)

### bowling Avg

In [77]:
player_df['bow_avg'] = np.round(player_df['runs_consided'] /player_df['wickets'],1)

### Dot Balls

In [78]:
dot_dict = bowling_df.groupby('bowlerName')['0s'].sum()
player_df['dots'] = player_df['name'].map(dot_dict)

### Dot ball %

In [79]:
player_df['dot_ball_%'] = np.round(player_df['dots']/player_df['balls_bowled']*100 , 1)

### maidens

In [80]:
maiden_dict = bowling_df.groupby('bowlerName')['maiden'].sum()
player_df['maidens'] = player_df['name'].map(maiden_dict)

### Avg balls faced

In [81]:
player_df['avg_ball_faced'] = np.round(player_df['total_balls']/player_df['innings'],1)

In [84]:
player_df.head(2)

,name,contry,battingStyle,bowlingStyle,playingRole,indivisual_runs,total_balls,innings,dissmissals,fours,...,bow_economy,balls_bowled,wickets,Bow_strike_rate,runs_consided,bow_avg,dots,dot_ball_%,maidens,avg_ball_faced
0,Maheesh Theekshana,Sri Lanka,Right hand Bat,Right arm Offbreak,Bowler,11.0,14.0,2.0,1.0,0.0,...,6.67375,181.0,9.0,20.1,202.0,22.4,71.0,39.2,0.0,7.0
1,Dushmantha Chameera,Sri Lanka,Right hand Bat,Right arm Fast,Bowler,9.0,16.0,2.0,1.0,0.0,...,6.83000,47.0,4.0,11.8,54.0,13.5,22.0,46.8,0.0,8.0


In [94]:
player_df.to_csv(config.CSV_FILE_PATH,index = False)                 # save df to csv file